In [6]:
!pip install -qU langchain langchain-core langchain-community langchain-experimental langchain-groq langchain-huggingface langchain-chroma langchain-text-splitters pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 6.6 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

In [43]:
from langchain_groq import ChatGroq


llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0,
)

In [4]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

332 M Parameter</br>
1.25 GB</br>
1024 Dimension</br>
512 Max Token

In [5]:
embeddings = HuggingFaceEmbeddings(model_name="katanemo/bge-large-en-v1.5")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

# Load the documents

In [7]:
file_path = '/content/drive/MyDrive/Colab_Notebooks/GenerativeAI/RAG/Hybrid_RAG/Knowledge/pdf/TourGuide.pdf'

In [8]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(file_path)
documents = loader.load()

# Chunk document

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [16]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=5000,
    chunk_overlap=800,
    length_function=len,
    is_separator_regex=False,
)

In [18]:
docs = text_splitter.split_documents(documents)

# Embeddings

In [12]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 1024, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='katanemo/bge-large-en-v1.5', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

# Vector Store

In [19]:
from langchain_chroma import Chroma

In [21]:
persist_directory="chroma_db"


if not os.path.exists(persist_directory):
  # save to disk
  db = Chroma.from_documents(docs, embeddings, persist_directory=persist_directory)
else:
  # load from disk
  db = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

# Retriever

In [22]:
retriever = db.as_retriever(search_kwargs={"k": 4})

# Prompt

In [24]:
from langchain_core.prompts import PromptTemplate

In [25]:
template = """Answer the question in only one sentence based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)


# Chain

In [27]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [28]:
retrieval_chain = (
    RunnableParallel({
        "context": retriever,
        "question": RunnablePassthrough()
      })
    | prompt
    | llm
    | StrOutputParser()
    )


In [30]:
result = retrieval_chain.invoke("Can i click pictures in front of Taj Mahal?")
result

'Yes, you can click pictures in front of the Taj Mahal, but photography is not allowed inside the main mausoleum.'

# Calculate Time

In [32]:
import time

In [44]:
start_time = time.time()

result = retrieval_chain.invoke("Can i click pictures in front of Taj Mahal?")
print(result)
end_time = time.time()

print(f"Time taken: {end_time - start_time} seconds")

Yes, you can click pictures in front of the Taj Mahal, but photography is not allowed inside the main mausoleum.
Time taken: 1.0816986560821533 seconds


# Async Invoke chain

In [45]:
# You must always use await
result = await retrieval_chain.ainvoke("Can i click pictures in front of Taj Mahal?")
result

'Yes, you can click pictures in front of the Taj Mahal, but photography is not allowed inside the main mausoleum.'

In [46]:
start_time = time.time()

result = await retrieval_chain.ainvoke("Can i click pictures in front of Taj Mahal?")
print(result)
end_time = time.time()

print(f"Time taken: {end_time - start_time} seconds")

Yes, you can click pictures in front of the Taj Mahal, but photography is not allowed inside the main mausoleum.
Time taken: 34.932597160339355 seconds


# Batch and Async Batch

In [48]:
start_time = time.time()

result = retrieval_chain.batch([
    "Can i click pictures in front of Taj Mahal?",
    "Who was Hampis King?"
    ])
print(result)
end_time = time.time()

print(f"Time taken: {end_time - start_time} seconds")

['Yes, you can click pictures in front of the Taj Mahal, but photography is not allowed inside the main mausoleum.', 'Hampi was the capital of the Vijayanagara Empire, so it had multiple kings throughout its history, including Harihara-I, Krishnadevaraya, and Sadasiva Raya.']
Time taken: 80.19152450561523 seconds


In [49]:
start_time = time.time()

result = await retrieval_chain.abatch([
    "Can i click pictures in front of Taj Mahal?",
    "Who was Hampis King?"
    ])
print(result)
end_time = time.time()

print(f"Time taken: {end_time - start_time} seconds")

['Yes, you can click pictures in front of the Taj Mahal, but photography is not allowed inside the main mausoleum.', 'Hampi was the capital of the Vijayanagara Empire, so it had multiple kings throughout its history, including Harihara-I, Krishnadevaraya, and Sadasiva Raya.']
Time taken: 73.28119587898254 seconds


# Passing Multiple Input

[GITHUB](https://github.com/langchain-ai/langchain/discussions/16421)

In [53]:
my_dict = {'Youtube': '@shubham10','Blog': "Shubham's blog"}
my_dict

{'Youtube': '@shubham10', 'Blog': "Shubham's blog"}

In [58]:
from operator import itemgetter

get_item = itemgetter('Blog')

In [59]:
get_item(my_dict)

"Shubham's blog"

In [61]:
from operator import itemgetter

This `"context": itemgetter("query") | retriever` form is required when trying to pass a dictionary to `invoke`

In [65]:
template = """Answer the question in just one sentence, in a given tone based only on the following context:
{context}

Question: {question}

Tone :{tone}
"""
prompt = PromptTemplate.from_template(template)

In [66]:
retrieval_chain = (
    RunnableParallel({
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "tone": itemgetter("tone")
      })
    | prompt
    | llm
    | StrOutputParser()
    )

In [67]:
response = retrieval_chain.invoke({
    'question': 'Who was Hampis King?',
    'tone': 'Humarous'
})
response

'It\'s like trying to find a needle in a haystack, but I\'ll give it a shot! From what I gathered, Hampi was a fortified city and the capital of the Vijayanagara empire in the 14th century, and it had a bunch of kings, but I\'m not exactly sure who the main king was. I mean, there was Harihara-I, Sadasiva Raya, and Krishnadevaraya, but I\'m not sure if any of them were the "king" of Hampi. Maybe it\'s like a game of "king of the castle" and they all took turns being the king? Anyway, I\'m just a humble AI, I don\'t have all the answers, but I\'ll keep searching for that needle!'

In [80]:
template = """Answer the question in just one sentence, in a given tone based only on the following context:
{context}


Question: {question}

Tone:{tone}
"""
prompt = PromptTemplate.from_template(template)

Order matters.

In [81]:
retrieval_chain = (
    RunnableParallel({
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "tone": itemgetter("tone")
      })
    | prompt
    | llm
    | StrOutputParser()
    )

In [82]:
response = retrieval_chain.invoke({
    'question': 'Can I click pictures in front of Taj Mahal?',
    'tone': 'Humarous'
})
response

"You want to click some snaps in front of the Taj Mahal, huh? Well, let me tell you, it's a no-go for taking pics inside the main mausoleum, but you can definitely take some fabulous photos in front of it, just don't forget to capture the stunning architecture and the lovely people in your shots, because, let's be honest, the real stars of the show are the tourists!"